In [1]:
import pandas as pd
import numpy as np

In [2]:
data_model = pd.read_csv('model_right.csv')
data_sour = pd.read_csv('annual+data+SIC.csv')
data_sour['datadate'] = data_sour.apply(lambda x: int(x['datadate']/10000), axis=1)#datadate取前四位

In [3]:
model_sic_group = data_model.groupby('SICCD')#data_wash processing
sour_sic_group = data_sour.groupby('sic')
def remove_unmatched_data(x):
    sic_index = x['sic']
    datadate_index = x['datadate']
#     print('sic of current rows:', datadate_index)
    model_index_list = list(model_sic_group.get_group(sic_index)['date'])
#     print('date of model with sic of ',sic_index,' ', model_index_list)
    if datadate_index in model_index_list:
        return True
    else:
        return False
data_sour['if_retained'] = data_sour.apply(remove_unmatched_data, axis=1) 
data_washed = data_sour.loc[data_sour['if_retained']==True]#只取保留的数据
# data_washed

In [7]:
group_sic = data_washed.groupby('sic')
def herf(x):#计算herf函数
    date_index = x['datadate']
    sic_index = x['sic']
    current_revt = x['revt']
#     print('sic', sic_index, 'revt', current_revt, 'datadate', date_index)
    t = group_sic.get_group(sic_index).groupby('datadate').sum()['revt']#统计sic下分date计算revt
#     print(t[date_index])
    if t[date_index] != 0 and current_revt!=np.nan:
#         print((x['revt']*100.0 / t[date_index])**2)
        return (x['revt']*100.0 / t[date_index])**2
    else:
        return np.nan
data_washed['herf'] = data_washed.apply(herf, axis= 1)#计算herf
# data_washed

/home/wdd/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [8]:
# data_washed

In [9]:
group_sic = data_washed.groupby('sic')
def leverage(x):#计算leverage
    date_index = x['datadate']
    sic_index = x['sic']
    current_dltt = x['dltt']
    current_at= x['at']
    current_mkvalt = x['mkvalt']
    t = group_sic.get_group(sic_index).groupby('datadate').sum()['mkvalt']#统计当前sic下不同date的mkvalt
#     print(t)
    if t[date_index] != 0 and current_at != 0:#此处判断分母不为0
        return current_dltt * current_mkvalt/ ( current_at * t[date_index])
    else:
        return np.nan
data_washed['leverage'] = data_washed.apply(leverage, axis=1)

/home/wdd/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [10]:
group_sic = data_washed.groupby('sic')
def profit(x):#计算profit
    date_index = x['datadate']
    sic_index = x['sic']
    current_revt = x['revt']
    current_oibdp = x['oibdp']
    current_mkvalt = x['mkvalt']
    t = group_sic.get_group(sic_index).groupby('datadate').sum()['mkvalt']
#     print(t)
    if t[date_index] != 0 and current_revt != 0:
        return current_oibdp * current_mkvalt/ ( current_revt * t[date_index])
    else:
        return np.nan
data_washed['profit'] = data_washed.apply(profit, axis=1)

/home/wdd/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [21]:
data_washed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36973 entries, 5 to 87140
Data columns (total 25 columns):
gvkey          36973 non-null int64
datadate       36973 non-null int64
fyear          36846 non-null float64
indfmt         36973 non-null object
consol         36973 non-null object
popsrc         36973 non-null object
datafmt        36973 non-null object
tic            36973 non-null object
cusip          36973 non-null object
conm           36973 non-null object
curcd          36846 non-null object
fyr            36846 non-null float64
at             35223 non-null float64
dltt           35093 non-null float64
oibdp          34919 non-null float64
revt           35094 non-null float64
cik            33980 non-null float64
costat         36973 non-null object
mkvalt         28579 non-null float64
sic            36973 non-null int64
spcsrc         18573 non-null object
if_retained    36973 non-null bool
herf           35094 non-null float64
leverage       28338 non-null float6

In [20]:
view_data = data_washed.dropna()#丢弃存在Nan的行
view_data.groupby('sic').mean()[['herf', 'leverage', 'profit']]#分sic平均

,herf,leverage,profit
sic,,,
2013,861.300870,0.046696,0.019984
2015,1389.689036,0.110110,-0.003860
2020,1466.991764,0.079537,0.013007
2030,558.501348,0.054412,0.028561
2040,121.047501,0.055714,0.029125
2060,132.752782,0.025461,0.030211
2086,115.647348,0.006712,0.019737
2111,652.191512,0.267811,0.233058
2221,2499.062878,0.125892,0.064488


In [13]:

# sort_sic = data_washed.groupby('sic')
# sort_revt_by_sic = sort_sic['revt'].sum()
# sort_mkvalt_by_sic = sort_sic['mkvalt'].sum()
# data_washed['herf'] = data_washed.apply(lambda x: (x['revt']*100.0 / sort_revt_by_sic[x['sic']])**2 if sort_revt_by_sic[x['sic']] != 0 else np.nan, axis= 1)
# data_washed['leverage'] = data_washed.apply(lambda x: (x['mkvalt'] * x['dltt'])/ (sort_mkvalt_by_sic[x['sic']] *x['at']) if x['at'] != 0 else np.nan, axis=1)
# data_washed['profit'] = data_washed.apply(lambda x: x['oibdp'] * x['mkvalt']/ (x['revt'] * sort_mkvalt_by_sic[x['sic']]) if x['revt'] else np.nan, axis=1)
# data_washed.to_csv('calculated_result.csv', index=True, header=True)
# view_data = data_washed.set_index(['sic', 'datadate'])[['herf', 'leverage', 'profit']]
# view_data =view_data.dropna(axis=0, how='any')
# view_data.to_csv('wanted_data.csv', index=True, header=True)

In [14]:
# sort = data_washed.groupby('sic')
# sort.get_group(2013).groupby('datadate').mean()[['herf', 'leverage', 'profit']]

In [16]:
sort_sic = view_data.groupby('sic')
# count =0
# sort_sic.get_group(2013)
for i,content in sort_sic:
    result = sort_sic.get_group(i).groupby('datadate').mean()[['herf', 'profit', 'leverage']]
#     print('current sic', i, result)
    result.to_csv('result/sic_'+str(i)+'.csv', header= True)#保存文件
#     t = content.groupby('datadate').mean()[['herf', 'leverage', 'profit']]
#     t
#     content.groupby('datadate').mean()